In [1]:
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
#from sklearn.model_selection import train_test_split

In [2]:
test_model = tf.keras.models.load_model(r'C:\Users\user\Pictures\model_two_lit')

In [3]:
# test_model.summary()

In [4]:
# 生成 dataset
test_dir = r"D:/independent_study/program/DL/picture/classify/b"

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode="rgb",
    batch_size=2,
    image_size=(224, 224),
    shuffle=False,
    seed=0,
    interpolation="bilinear",
    follow_links=False,
)

# 像素 0~255 -> 0~1

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255)
test_dataset = test_dataset.map(lambda x: (normalization_layer(x)))


Found 16 files belonging to 1 classes.


In [5]:
test_dataset

<MapDataset element_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None)>

In [6]:
# 預測分類
y_predict = test_model.predict(test_dataset)

8/8 [==============================] - 4s 11ms/step


In [7]:
y_predict

array([[0.05522799, 0.944772  ],
       [0.02983271, 0.97016734],
       [0.13749681, 0.86250323],
       [0.02519871, 0.9748013 ],
       [0.04680618, 0.95319384],
       [0.02334955, 0.9766505 ],
       [0.01834072, 0.98165923],
       [0.03215788, 0.9678421 ],
       [0.01608356, 0.98391646],
       [0.3139913 , 0.68600875],
       [0.2827866 , 0.7172134 ],
       [0.0633465 , 0.9366535 ],
       [0.12425265, 0.8757474 ],
       [0.07743286, 0.9225672 ],
       [0.2725596 , 0.72744036],
       [0.13483866, 0.86516136]], dtype=float32)

In [8]:
# 選取機率最大的分類
y_predict_lable = np.argmax(y_predict, axis=1)
y_predict_lable

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [9]:
#未知分類

In [10]:
def get_c_v(p_s='', class_num=2):
    c_v = []  # 置信度向量
    for label in range(class_num):
        y_predict_lable = np.argmax(p_s, axis=1)
        y_index = np.argwhere(y_predict_lable==label).reshape(-1)
        p_temp = p_s[y_index]
        p_temp = np.array(p_temp)
        if len(np.shape(p_temp))<=1 or np.size(p_temp)<=1:
            c = 1.0
        else:
            IQR = np.percentile(p_temp[:, label], 75) - np.percentile(p_temp[:, label], 25)
            c = np.percentile(p_temp[:, label], 25) - 1.5*IQR
        c_v.append(c)
    return c_v

In [11]:
c = get_c_v(p_s = y_predict, class_num=2)
c

[1.0, 0.7042533233761787]

In [12]:
def p_to_label(p_s='',c_v=""):
    y_predict_lable = np.argmax(p_s, axis=1)
    y_out = np.zeros(y_predict_lable.shape)
    for label in range(p_s.shape[1]):
        y_index = np.argwhere(y_predict_lable==label).reshape(-1)
        p_temp = y_predict[y_index][:,label]
        p_temp = np.where(p_temp<c[label] ,-1 , label)
        y_out[y_index] = p_temp
    return y_out.astype(int)

y_out = p_to_label(p_s=y_predict,c_v=c)
y_out

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1])

In [13]:
from collections import Counter

value = Counter(y_out).most_common()
value
# type(value)

[(1, 15), (-1, 1)]

In [14]:
# import operator

max_key = max(dict(value), key = dict(value).get)
max_key

1

                        ############ 很重要!!! 記得儲存每次ㄉ模型紀錄 ############